<a href="https://colab.research.google.com/github/theotherpablo/ws-eng-conduit-ai-assessment/blob/master/Solving_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Business Problems with AI

## Objective
Develop a proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalog information and stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

## Instructions

- All requirements have to be implemented with the use of LLMs to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Because solving this assessment requires using advanced LLM capabilities, we provide you with a temporary OpenAI API key granting access to GPT-4o. You can use this key or your own, but please note that the provided key has a token quota, so use it wisely. We have carefully adjusted the limits to ensure they are sufficient for you to complete the task.
- Address the requirements in the order listed. Please review the requirements ahead to have a general implementation plan in place before you start.
- Your deliverables should include the code developed within this notebook, and a spreadsheet containing results across separate sheets. Comments detailing your thought process are appreciated.
- You are allowed to use additional libraries (langchain, etc.) in order to make solution more concise.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, create a new order line with the status **created**. If the order cannot be fully fulfilled, create a line with the status **out of stock**. After placing the order, update the stock to accurately reflect the current inventory levels.

    **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate and save response emails based on order processing results. Depending on the order status email should inform customer that their order was processed or could not be fulfilled. If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock. Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

    **Output**: Populate the **order-response** sheet with columns: email ID, response.

### 3. Handle product inquiry

Customers may ask general open questions.
- Respond to product inquiries using the information from the product catalog.
- Ensure that solution you provide scales well to handle the full catalog of over 100,000 products. This means that solutions which include raw product catalog in the prompt will be rated lower!

**Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria

- **Utilization of AI Tools**: Effectiveness in leveraging AI technologies provided (e.g., OpenAI API) to fulfill the task requirements. Employ your knowledge of various AI driven development techniques depending on the task.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational.
- **Code Quality and Clarity**: Code should be well-organized with clear logic and structure.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the spreadsheet. Double check before submitting!

We look forward to seeing your solution and how you approach solving real-world problems using AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [3]:
# Install the OpenAI Python package.
%pip install openai

In [5]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001L8hZMAS'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [6]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('email_id').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('subject').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('message').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['subject'].value_counts()
    for x_label, grp in _df_3.groupby('email_id')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('email_id')
_ = plt.ylabel('subject')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['message'].value_counts()
    for x_label, grp in _df_4.groupby('subject')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('subject')
_ = plt.ylabel('message')

# Task 1. Classify emails

In [25]:
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# prompt: classify each email inside emails_df into two categories: "product inquiry" and "order request" based on the subject field. If emails_df include the word "order" or "ordering" then classify it as "order request" otherwise add it to "product inquiry"

# Handle potential non-string values in the 'subject' column
emails_df['category'] = emails_df['subject'].apply(lambda x: 'order request' if isinstance(x, str) and ('order' in x.lower() or 'ordering' in x.lower()) else 'product inquiry')
#display(emails_df.head(email_id, category))
display(emails_df[['email_id', 'category']].head())

,email_id,category
0,E001,product inquiry
1,E002,product inquiry
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry


# Task 2. Process order requests

# Task 3. Handle product inquiry